In [1]:
#!/usr/bin/env python3
import sys
srcdir = "/project/mchaisso_100/cmb-16/tsungyul/work/vntr/danbing-tk/script/"
sys.path.insert(0, srcdir)

In [2]:
import numpy as np
import pandas as pd
import vntrutils as vu
import utils
# import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
# %load_ext autoreload
# %autoreload 2

In [3]:
from collections import defaultdict
import pickle
# import networkx as nx
# import pygraphviz
# import collections
import gc
# import glob
# import warnings
import statsmodels.api as sm
# from statsmodels.stats.multitest import fdrcorrection as fdr
# from scipy import stats
import gzip
# from sklearn.decomposition import PCA
# from scipy.stats import chi2
# import re
# import json

In [4]:
matplotlib.rc('font', size=7)
matplotlib.rc('axes', titlesize=7)
matplotlib.rc('xtick', labelsize=5)
matplotlib.rc('ytick', labelsize=5)
%load_ext autoreload
%autoreload 2

In [5]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/clinic/cdbg/v1/ki_tr.ccki_tr.pickle", 'rb') as f:
        ki_tr, ccki_tr = pickle.load(f)
    return ki_tr, ccki_tr
ki_tr, ccki_tr = test()

In [6]:
def get_ki(tri):
    si = ki_tr[tri-1] if tri else 0
    ei = ki_tr[tri]
    return si, ei
def get_ccki(tri):
    si = ccki_tr[tri-1] if tri else 0
    ei = ccki_tr[tri]
    return si, ei

# Bias correction for Geuvadis genotypes

In [7]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/clinic/bias/invariant_kmer/ikmer.ikstats.pickle", 'rb') as f:
        ikmer, ikstats = pickle.load(f)
    return ikmer, ikstats
ikmer, ikstats = test()

In [9]:
cov = np.loadtxt("/project/mchaisso_100/cmb-17/vntr_genotyping/analysis/read_depth/ctrl/Geuvadis_1kgp3.cov.tsv", usecols=1)

In [10]:
def test():
    ng = cov.size
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/clinic/cdbg/v1/tr_cck_ns.pickle", 'rb') as f:
        tr_cck_ns = pickle.load(f)
    nk = ki_tr[-1]
    ntr = len(ki_tr)
    
    NB = 10
    BS = ng // NB
    bias = np.full([ng, ntr], np.nan)
    for i in range(NB):
        BS_ = BS if i != NB-1 else ng - BS*i
        i0s = i * BS
        i0e = i0s + BS_
        print(f"batch {i}: loading gt", end="")
        with open(f"/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/1kg/Geuvadis/gt_pickle/gt.{i}.pickle", 'rb') as f:
            agt = pickle.load(f).astype(float) / cov[i0s:i0e,None]
        for tri in range(ntr):
            if tri % 1000 == 0:
                print(".", end="")
            si, ei = get_ki(tri)
            inv = np.nonzero(ikmer[si:ei])[0] + si
            if inv.size:
                bias[i0s:i0e, tri] = np.mean(agt[:,inv] / ikstats[0,inv], axis=1)
        print()
        # del agt
        # gc.collect()
    bias[bias==0] = np.nan
    bias /= np.nanmean(bias, axis=0)
    n0 = np.sum(~np.isfinite(bias))
    print(f"{n0} cells of nan's")
    return bias

bias = test()

batch 0: loading gt.................................................................................
batch 1: loading gt.................................................................................
batch 2: loading gt.................................................................................
batch 3: loading gt.................................................................................
batch 4: loading gt.................................................................................
batch 5: loading gt.................................................................................
batch 6: loading gt.................................................................................
batch 7: loading gt.................................................................................
batch 8: loading gt.................................................................................
batch 9: loading gt........................................................................

/tmp/SLURM_10003803/ipykernel_36800/1755666370.py:29: RuntimeWarning: Mean of empty slice
  bias /= np.nanmean(bias, axis=0)


In [11]:
pd.DataFrame(bias)

,0,1,2,3,4,5,6,7,8,9,...,80508,80509,80510,80511,80512,80513,80514,80515,80516,80517
0,0.967244,0.886937,1.001814,0.993814,0.824581,NaN,0.921442,0.935988,NaN,1.014504,...,0.778257,0.590820,0.647960,0.561766,0.695875,0.662091,0.748786,0.660592,0.722163,0.518526
1,1.100264,0.740333,0.990077,0.933627,1.285651,NaN,0.937583,0.630503,NaN,1.162598,...,1.392773,1.448627,1.383578,1.175435,1.129644,1.467819,1.457932,1.435576,1.340760,0.676278
2,0.712921,0.674776,0.934647,1.034726,1.199768,NaN,1.156937,1.263985,NaN,0.963124,...,1.342889,1.198836,1.115940,1.023365,1.461003,1.231081,1.517132,1.174494,1.411697,0.889540
3,1.060288,0.810742,1.103858,0.774385,1.211566,NaN,1.152162,0.787592,NaN,0.887865,...,1.115852,1.327815,1.260403,1.503500,1.367375,1.351580,1.195744,1.220166,1.074164,1.115250
4,0.996678,1.024603,0.985875,1.151266,1.325045,NaN,0.832522,0.946831,NaN,0.995653,...,0.762014,0.749178,0.673252,0.582571,0.785504,0.753785,0.761657,0.590411,0.748532,0.564284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,1.035126,1.113540,0.817988,1.009474,1.125360,NaN,0.893981,0.640699,NaN,1.075268,...,0.808904,0.714860,0.820519,0.776368,0.724129,0.692123,0.917543,0.556469,0.923541,0.387084
441,0.877939,1.182878,1.202039,1.059657,1.227311,NaN,1.069905,0.987188,NaN,1.327541,...,0.823498,0.826264,0.704047,0.838097,0.777129,0.771425,0.597507,0.781412,0.689313,0.859543
442,1.083293,1.084655,1.173789,1.186859,1.102744,NaN,1.256467,1.580770,NaN,1.202259,...,1.199546,1.152863,1.511450,1.405263,1.260736,1.216052,1.020014,1.708166,1.230403,1.556353
443,1.012700,0.818233,1.210631,1.043407,0.714698,NaN,1.225162,0.834320,NaN,0.941611,...,1.517673,1.480422,1.385798,1.217915,1.229372,1.291719,1.290491,1.520427,1.127654,1.073870


In [12]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/1kg/Geuvadis/bias/bias.v1.pickle", 'wb') as f:
        pickle.dump(bias, f)
# test()

In [17]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/1kg/Geuvadis/gt_pickle/acgt.pickle", 'rb') as f:
        return pickle.load(f).astype(float) # ng x ncck
acgt = test()

In [18]:
def test():
    ntr = len(ccki_tr)
    acgt_bc = np.copy(acgt)
    for tri in range(ntr):
        si, ei = get_ccki(tri)
        acgt_bc[:,si:ei] /= (bias[:,tri][:,None])
    return acgt_bc
acgt_bc = test()

In [19]:
pd.DataFrame(acgt_bc)

,0,1,2,3,4,5,6,7,8,9,...,4456871,4456872,4456873,4456874,4456875,4456876,4456877,4456878,4456879,4456880
0,1.996966,0.896799,3.108148,2.723844,3.200357,0.002471,1.379000,0.357316,0.844730,0.0,...,0.000000,0.369115,0.000000,1.094214,3.515527,1.806134,2.554079,0.663876,1.759272,1.687352
1,1.834290,1.121819,3.188245,2.681688,3.531503,0.000000,1.315783,0.862487,1.165309,0.0,...,0.000000,0.276255,0.295367,0.547107,2.237062,1.219710,1.668911,0.331938,1.053350,1.222639
2,1.891646,0.900375,2.924419,2.820544,3.087469,0.000000,1.784934,0.130460,0.465564,0.0,...,0.000000,0.281463,0.302413,0.547107,2.046077,1.090896,1.449223,0.331938,0.958291,1.003030
3,2.068542,0.904728,3.191762,2.679837,3.438544,0.000000,1.259139,0.326151,0.917698,0.0,...,0.000000,0.363444,0.244403,0.547107,1.649632,0.850998,1.164088,0.331938,0.816015,0.836761
4,1.771928,1.053244,2.833432,2.868432,2.796634,0.000000,0.961350,0.377673,0.630322,0.0,...,0.012202,0.671104,0.536883,1.094214,3.214925,1.847438,2.638482,0.663876,1.695438,1.480955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,1.672111,1.243904,2.984581,2.788880,2.951480,0.000000,0.883467,0.269846,0.621430,0.0,...,0.000000,0.679868,0.564419,1.094214,3.202029,2.421196,3.085795,0.663876,1.180224,1.641250
441,2.152797,0.933681,3.112614,2.721494,3.111957,0.000000,0.774777,0.154422,0.457184,0.0,...,0.000000,0.669400,0.649115,1.094214,1.505457,0.724623,1.352340,0.663876,0.811404,0.663876
442,1.936467,0.973050,3.049212,2.754863,3.210804,0.000000,1.068050,0.284538,1.071446,0.0,...,0.000000,0.350552,0.293272,0.547107,1.160869,0.494653,0.770331,0.331938,0.567312,0.585921
443,1.622746,1.085141,2.969736,2.796693,3.100339,0.000000,1.120647,0.252013,0.596045,0.0,...,0.000000,0.347328,0.301018,0.547107,1.208724,0.619762,0.932295,0.331938,0.607324,0.590112


In [20]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/1kg/Geuvadis/gt_pickle/acgt_bc.v1.pickle", 'wb') as f:
        pickle.dump(acgt_bc, f, protocol=pickle.HIGHEST_PROTOCOL)
# test()

In [ ]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/1kg/Geuvadis/gt_pickle/acgt_bc.v1.pickle", 'rb') as f:
        return pickle.load(f)
acgt_bc = test()

In [ ]:
def test():
    ntr = len(ki_tr)
    for tri in range(ntr):
        si, ei = get_ccki(tri)
        xs = np.sum(acgt_bc[:,si:ei], axis=1)
        utils.hist(xs, "Read TR dosage", "Count", f"{tri}", bins=50)
        input()
test()